In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

path = '/kaggle/input/col774-2022'
train_x_path = path + '/train_x.csv'
train_y_path = path + '/train_y.csv'
test_x_path = path + '/non_comp_test_x.csv'
test_y_path = path + '/non_comp_test_y.csv'

/kaggle/input/col774-2022/train_x.csv


In [3]:
train_x_data = ((pd.read_csv(train_x_path))['Title'])
train_y_data = ((pd.read_csv(train_y_path))['Genre'])
test_x_data = ((pd.read_csv(test_x_path))['Title'])
test_y_data = ((pd.read_csv(test_y_path))['Genre'])

total_x_data = pd.concat([train_x_data,test_x_data], axis = 0)
total_y_data = pd.concat([train_y_data, test_y_data], axis = 0)
model_name = "roberta-large"

In [5]:
data_dict_list = {}
data_dict_list["label"] = total_y_data
data_dict_list["text"] = total_x_data

In [6]:
from datasets import Dataset
data_train = Dataset.from_dict(data_dict_list)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [9]:
tokenized_train = data_train.map(preprocess_function, batched=True)

  0%|          | 0/40 [00:00<?, ?ba/s]

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=30)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    save_strategy = "no"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_train,
    tokenizer=tokenizer,
)

In [13]:
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 39900
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8729
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.017500
1000,1.517300
1500,1.339600
2000,1.225200
2500,1.172100
3000,0.930500
3500,0.940500
4000,0.818500
4500,0.725000
5000,0.711200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8729, training_loss=0.8286540756548977, metrics={'train_runtime': 6530.3131, 'train_samples_per_second': 42.77, 'train_steps_per_second': 1.337, 'total_flos': 1.980583004666683e+16, 'train_loss': 0.8286540756548977, 'epoch': 7.0})

In [14]:
trainer.save_model()

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin
tokenizer config file saved in ./results/tokenizer_config.json
Special tokens file saved in ./results/special_tokens_map.json


In [15]:
import numpy as np

In [16]:
def accuracy(preds, labels):
    correct = 0
    for i in range(len(preds)):
        if(preds[i] == labels[i]):
            correct += 1
    return correct/len(preds)*100

In [17]:
competition_x = (pd.read_csv(path + '/comp_test_x.csv'))['Title']
final_dict_list = {}
final_dict_list["label"] = [0 for i in range(len(competition_x))]
final_dict_list["text"] = competition_x
data_final = Dataset.from_dict(final_dict_list)
tokenized_final = data_final.map(preprocess_function, batched=True)
predictions = trainer.predict(tokenized_final)

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 11514
  Batch size = 32


In [18]:
preds = np.argmax(predictions.predictions, axis=-1)

[ 8 20 27 ... 23 12  4]


In [19]:
id_value = []
genre = []

for i in range(len(preds)):
    id_value.append(i)
    genre.append(preds[i])

dictCSV = {"Id" : id_value, 'Genre':genre}
df1 = pd.DataFrame(dictCSV)
df1.to_csv('comp_test_y.csv',index = False)